![Constantine Quantum Technologies for QFF22-Algiers](imgs/qff22_cqtech_banner.png)

# QKD challenge - Qiskit Fall Fest 2022, Algiers

### The BB84 Protocol (from Qiskit)

To make things easy on you, here's the complete code for the BB84 protocol. You may copy and modify it for your solutions bellow.

In [1]:
!pip install qiskit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 16.5 MB/s 
     |████████████████████████████████| 19.2 MB 245 kB/s 
     |████████████████████████████████| 240 kB 57.5 MB/s 
     |████████████████████████████████| 55 kB 2.9 MB/s 
     |████████████████████████████████| 112 kB 25.1 MB/s 
     |████████████████████████████████| 50 kB 5.2 MB/s 
     |████████████████████████████████| 49 kB 3.0 MB/s 
     |████████████████████████████████| 37.5 MB 47 kB/s 
     |████████████████████████████████| 943 kB 55.4 MB/s 
     |████████████████████████████████| 4.0 MB 42.2 MB/s 
     |████████████████████████████████| 1.9 MB 52.2 MB/s 
     |████████████████████████████████| 112 kB 58.2 MB/s 
  Created wheel for qiskit: filename=qiskit-0.39.0-py3-none-any.whl size=12188 sha256=323fbc8284038614fe43446d71fa87a0cc7f998e3041b16ab515a74e60b5e1ad
  Stored in directory: /root/.cache/pip/wheels/63/d0/9a/25714

In [2]:
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from numpy.random import randint
import numpy as np

In [37]:
qc=QuantumCircuit(1,1)
qc.h(0)
a=qc
qc.measure(0,0)
aer_sim = Aer.get_backend('aer_simulator')
qobj = assemble(qc, shots=1, memory=True)
result = aer_sim.run(qobj).result()
measured_bit = int(result.get_memory()[0])
measured_bit
print(a,qc)

   ┌───┐┌─┐
q: ┤ H ├┤M├
   └───┘└╥┘
c: ══════╩═
               ┌───┐┌─┐
q: ┤ H ├┤M├
   └───┘└╥┘
c: ══════╩═
           


In [8]:
def encode_message(bits, bases):
    message = []
    for i in range(n):
        qc = QuantumCircuit(1,1)
        if bases[i] == 0: # Prepare qubit in Z-basis
            if bits[i] == 0:
                pass 
            else:
                qc.x(0)
        else: # Prepare qubit in X-basis
            if bits[i] == 0:
                qc.h(0)
            else:
                qc.x(0)
                qc.h(0)
        qc.barrier()
        message.append(qc)
        
    return message

In [4]:
def measure_message(message, bases):
    backend = Aer.get_backend('aer_simulator')
    measurements = []
    for q in range(n):
        if bases[q] == 0: # measuring in Z-basis
            message[q].measure(0,0)
        if bases[q] == 1: # measuring in X-basis
            message[q].h(0)
            message[q].measure(0,0)
        aer_sim = Aer.get_backend('aer_simulator')
        qobj = assemble(message[q], shots=1, memory=True)
        result = aer_sim.run(qobj).result()
        measured_bit = int(result.get_memory()[0])
        measurements.append(measured_bit)
    return measurements

In [5]:
def remove_garbage(a_bases, b_bases, bits):
    good_bits = []
    for q in range(n):
        if a_bases[q] == b_bases[q]:
            # If both used the same basis, add
            # this to the list of 'good' bits
            good_bits.append(bits[q])
    return good_bits

In [6]:
def sample_bits(bits, selection):
    sample = []
    for i in selection:
        # use np.mod to make sure the
        # bit we sample is always in 
        # the list range
        i = np.mod(i, len(bits))
        # pop(i) removes the element of the
        # list at index 'i'
        sample.append(bits.pop(i))
    return sample

In [9]:
np.random.seed(seed=3)  # We use a known seed for RNG to make the results reproducible.
n = 100

## Step 1: Alice generates random bits.
alice_bits = randint(2, size=n)
## Step 2: Alice chooses a series of random bases: one for each bit.
alice_bases = randint(2, size=n)

## Step 3: Alice then sends a quantum message with her bits encoded in her random bases.
message = encode_message(alice_bits, alice_bases)

## Step 4: Bob chooses random bases of his own.
bob_bases = randint(2, size=n)
## Step 5: Bob then measures Alice's message in his own bases.
bob_results = measure_message(message, bob_bases)

## Step 6: Alice and Bob make their bases public, compare them, and only keep
#          the measurements where they used the same bases.
bob_key = remove_garbage(alice_bases, bob_bases, bob_results)
alice_key = remove_garbage(alice_bases, bob_bases, alice_bits)


## Step 5
sample_size = 15
bit_selection = randint(n, size=sample_size)
bob_sample = sample_bits(bob_key, bit_selection)
print("  bob_sample = " + str(bob_sample))
alice_sample = sample_bits(alice_key, bit_selection)
print("alice_sample = "+ str(alice_sample))
print("-----------")
print("  bob_key = " + str(bob_key))
print("alice_key = "+ str(alice_key))



  bob_sample = [0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1]
alice_sample = [0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1]
-----------
  bob_key = [0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1]
alice_key = [0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1]


___________________________________________________________________________________

# YOUR SOLUTIONS

### 1. PNS attack on BB84

In practice, Alice will not send single photons/qubits every single time. She will likely send multiple each time. This is because it's hard to reliably generate single photons.

In this task, Alice will always send exactly 2 photons/qubits each time.
How can Eve take advantage of this to retrieve Alice and Bob's secrete key without them knowing? Simulate this scenario using Qiskit.

In [ ]:
def encode_message(bits, bases):
    message = []
    for i in range(n):
        qc = QuantumCircuit(1,1)
        if bases[i] == 0: # Prepare qubit in Z-basis
            if bits[i] == 0:
                pass 
            else:
                qc.x(0)
        else: # Prepare qubit in X-basis
            if bits[i] == 0:
                qc.h(0)
            else:
                qc.x(0)
                qc.h(0)
        qc.barrier()
        message.append(qc)
        
    return message

In [57]:
def measure_message(message, bases):
    backend = Aer.get_backend('aer_simulator')
    measurements = []
    q=0
    while q<50 :
        if bases[q] == 0: # measuring in Z-basis
            message[2*q].measure(0,0)
        if bases[2*q] == 1: # measuring in X-basis
            message[2*q].h(0)
            message[2*q].measure(0,0)
        aer_sim = Aer.get_backend('aer_simulator')
        qobj = assemble(message[q], shots=1, memory=True)
        result = aer_sim.run(qobj).result()
        measured_bit = int(result.get_memory()[0])
        measurements.append(measured_bit)
        q=q+1
    return measurements

In [ ]:
def remove_garbage(a_bases, b_bases, bits):
    good_bits = []
    for q in range(n):
        if a_bases[q] == b_bases[q]:
            # If both used the same basis, add
            # this to the list of 'good' bits
            good_bits.append(bits[q])
    return good_bits

In [ ]:
def sample_bits(bits, selection):
    sample = []
    for i in selection:
        # use np.mod to make sure the
        # bit we sample is always in 
        # the list range
        i = np.mod(i, len(bits))
        # pop(i) removes the element of the
        # list at index 'i'
        sample.append(bits.pop(i))
    return sample

In [58]:
np.random.seed(seed=3)
n = 100  # Using 100 quantum states
#####   DO NOT CHANGE THE CODE ABOVE THIS LINE!   #####

## Step 1: Alice generates random bits.
pre_alice_bits = randint(2, size=50)

## Step 2: Alice chooses a series of random bases: one for each bit.
pre_alice_bases = randint(2, size=50)
alice_bits = []
alice_bases = []
a=0
b=0
while(a<50):

  
  alice_bits.append(pre_alice_bits[a]);
  
  alice_bits.append(pre_alice_bits[a])

  alice_bases.append(pre_alice_bases[a])
  alice_bases.append(pre_alice_bases[a])
  a=a+1
  b=b+2


## Step 3: Alice then sends a quantum message with her bits encoded in her random bases.
message = encode_message(alice_bits, alice_bases)

## Interception
eve_bases = randint(2, size=n)
eve_results = measure_message(message, eve_bases)

## Step 4: Bob chooses random bases of his own.
bob_bases = randint(2, size=n)
## Step 5: Bob then measures Alice's message in his own bases.
bob_results = measure_message(message, bob_bases)

## Step 6: Alice and Bob make their bases public, compare them, and only keep
#          the measurements where they used the same bases.
bob_key = remove_garbage(alice_bases, bob_bases, bob_results)
alice_key = remove_garbage(alice_bases, bob_bases, alice_bits)


#####   DO NOT CHANGE THE CODE BELLOW THIS LINE!   #####

# RESULTS -------------------------------------
print("  bob_sample = " + str(bob_sample))
print("alice_sample = "+ str(alice_sample))
print("  eve_sample = "+ str(eve_sample))
print("-----------")
print("  bob_key = " + str(bob_key))
print("alice_key = "+ str(alice_key))
print("  eve_key = " + str(eve_key))


QiskitError: ignored

#### Explain your solution (Optional)

If you did not have time to implement your code, or simply want to add further explanations, you can describe your solution in the cell bellow.<br>
(in both cases you are still expected to submit code!)

Solution:
For every bit we send 2 states, meaning if the message was:
[1,0,1,0,1,1]
the states will be
[1,1,0,0,1,1,0,0,1,1,1,1] (with the state notation)
so every bit is doubled meaning that a message of 50 bits will be sent in the form of 100 states, so when Bob recieves the message, he will only read the pair index states (meaning every other states, in other ones: one by one) for example Bob will read the states indexed 0,2,4,6...
when Eve intervenes she will measure the states indexed with impair numbers for example 1,3,5,7.... and since every bit is doubled in state, when Bob measures the states he will not notice any difference since he is measuring the impair states while Eve measured the pair ones

______________________________

### 2. Implementing BBM92 using Qiskit

Using the functions defined above, and creating new ones, implement the BBM92 protocol.

**Reminder:** In the BBM92 protocol, a pair of entangled photons/qubits is generated. Alice receives one, and Bob receives the other.

In [ ]:
# Your function(s) here ...   


In [ ]:
# Your function(s) here ...   


In [ ]:
np.random.seed(seed=3)
n = 100  # Using 100 quantum states
#####   DO NOT CHANGE THE CODE ABOVE THIS LINE!   #####

# ...
# ...    Your solution here    ...
# ...

#####   DO NOT CHANGE THE CODE BELLOW THIS LINE!   #####

# RESULTS -------------------------------------
print("  bob_sample = " + str(bob_sample))
print("alice_sample = "+ str(alice_sample))
print("-----------")
print("  bob_key = " + str(bob_key))
print("alice_key = "+ str(alice_key))


#### Explain your solution (Optional)

If you did not have time to implement your code, or simply want to add further explanations, you can describe your solution in the cell bellow.<br>
(in both cases you are still expected to submit code!)

*\[REPLACE THIS WITH YOUR ANSWER\]*

*(double click here to modify)*

________________________________________________________

### 3. BBM92 with interception of the duplicate photons

For the same reasons as in the 1st task (with BB84), 2 pairs of photons/qubits will be generated each time instead of 1.<br>
In this scenario, Alice and Bob will receive their respective photons as in the 2nd task, and Eve will receive one of the photons from the second pair.

Show that, unlike in the BB84 case, Eve does not get any information from the additional photons after Alice and Bob make their bases public.

In [ ]:
# Your function(s) here ...   


In [ ]:
# Your function(s) here ...   


In [ ]:
np.random.seed(seed=3)
n = 100  # Using 100 quantum states
#####   DO NOT CHANGE THE CODE ABOVE THIS LINE!   #####

# ...
# ...    Your solution here    ...
# ...

#####   DO NOT CHANGE THE CODE BELLOW THIS LINE!   #####

# RESULTS -------------------------------------
print("  bob_sample = " + str(bob_sample))
print("alice_sample = "+ str(alice_sample))
print("  eve_sample = "+ str(eve_sample))
print("-----------")
print("  bob_key = " + str(bob_key))
print("alice_key = "+ str(alice_key))
print("  eve_key = " + str(eve_key))  # Shoud be different than Alice and Bob's


#### Explain your solution (Optional)

If you did not have time to implement your code, or simply want to add further explanations, you can describe your solution in the cell bellow.<br>
(in both cases you are still expected to submit code!)

*\[REPLACE THIS WITH YOUR ANSWER\]*

*(double click here to modify)*

________________________________________________________